Chapter 6: Error Handling
-----------------------------

When it comes to runtime errors, developers traditionally chose one of these two approaches:

1. Ignore the problem (Not recommended for production)
2. Check all possible error conditions (known as LBYL – Look Before You Leap)



1. Ignore the problem (Not recommended for production):
   - No error handling (fragile)

In [ ]:
# Reads a number from a tab-separated file and writes 20% of it to another file
with open('myfile.csv') as fh:
    line = fh.readline()
    value = line.split('\t')[0]
    with open('other.txt', "w") as fw:
        fw.write(str(int(value) * 0.2))
#FileNotFoundError: [Errno 2] No such file or directory: 'myfile.csv'

FileNotFoundError: [Errno 2] No such file or directory: 'myfile.csv'

2. Check all possible error conditions (known as LBYL – Look Before You Leap)
 - ❌ Cons:
Harder to read + Logic is mixed with error checking

In [ ]:
import os

iname = input("Enter input filename: ")
oname = input("Enter output filename: ")

if os.path.exists(iname):
    with open(iname) as fh:
        line = fh.readline()
        if "\t" in line:
            value = line.split('\t')[0]
            if os.access(oname, os.W_OK) == 0:
                if value.isdigit():
                    with open(oname, 'w') as fw:
                        fw.write(str(int(value) * 0.2))
                else:
                    print("Can’t be converted to int")
            else:
                print("Output file is not writable")
        else:
            print("No TAB found in the input line")
else:
    print("Input file does not exist")


Enter input filename: hola
Enter output filename: hello
The file doesn’t exist


3. 🔸 Better Alternative: EAFP – Easier to Ask Forgiveness than Permission
   - In Python, try, except, else, and finally blocks are used for this.

- try: Wraps the risky code
- except: Runs only if an error is raised
- else: Runs if no error happens in the try block (optional)
- finally: Always runs, used for clean-up (optional).


In [ ]:
try:
    # Error-prone code
except:
    # Error handling code
[else:
    # Executes only if try block does NOT raise an exception
finally:
    # Always executes, for clean-up actions]


In [1]:
try:
    print(0 / 0)
except:
    print("Houston, we have a problem...")
# Output: Houston, we have a problem...


Houston, we have a problem...


#### Improved Version of File Handling with Try/Except (Modern Python)


In [2]:
try:
    # Get filenames from user
    iname = input("Enter input filename: ")
    oname = input("Enter output filename: ")

    # Read first line from input file
    with open(iname) as fh:
        line = fh.readline()

    # Extract value if tab exists
    if '\t' in line:
        value = line.split('\t')[0]

        # Write to output file
        with open(oname, 'w') as fw:
            fw.write(str(int(value) * 0.2))
except FileNotFoundError:
    print("Input file does not exist.")
except PermissionError:
    print("No permission to write the output file.")
except ValueError:
    print("The value can't be converted to int.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
else:
    print("Thank you! Everything went OK.")


Enter input filename:  myfile.csv
Enter output filename:  other.txt


Input file does not exist.


- In Jupyter Notebook, an EOFError does not normally occur because Jupyter always waits for input and does not easily reach the end-of-file (EOF) condition.
This error is more commonly encountered when running scripts in a terminal.

- If the program is run in a terminal and the user does not enter any input but instead presses:

- Ctrl+D on macOS/Linux

- Ctrl+Z followed by Enter on Windows

- then the program raises an EOFError and jumps to the except EOFError: block, printing:

#### Nested Try/Except Blocks for Better Error Separation

In [3]:
# Get input and output filenames
iname = input("Enter input filename: ")
oname = input("Enter output filename: ")

try:
    with open(iname) as fh:
        line = fh.readline()
except FileNotFoundError:
    print("Input file does not exist.")
else:
    if '\t' in line:
        value = line.split('\t')[0]
        try:
            with open(oname, 'w') as fw:
                fw.write(str(int(value) * 0.2))
        except PermissionError:
            print("No permission to write the output file.")
        except ValueError:
            print("The value can't be converted to int.")
        else:
            print("Thank you! Everything went OK.")
    else:
        print("There is no TAB. Check the input file.")


Enter input filename:  myfile.csv
Enter output filename:  other.txt


Input file does not exist.


#### Exception Types
Exceptions in Python are not all the same. For example, using an undefined variable raises a NameError, while combining incompatible data types raises a TypeError.

👉 You can find the full list of built-in exceptions here: https://docs.python.org/3/library/exceptions.html


- How to Handle Different Exceptions
- You can catch all exceptions using a generic except, but that’s not recommended:

In [ ]:
# Generic error handling (not recommended)
d = {"A": "Adenine", "C": "Cytosine", "T": "Thymine", "G": "Guanine"}

try:
    print(d[input("Enter letter: ")])
except:
    print("No such nucleotide")


Enter letter: L
No such nucleotide


- Why it's bad: It hides unexpected bugs, such as EOF input. Better to handle specific errors:

In [ ]:
# Handling specific exceptions
d = {"A": "Adenine", "C": "Cytosine", "T": "Thymine", "G": "Guanine"}

try:
    print(d[input("Enter letter: ")])
except EOFError:
    print("Good bye!")
except KeyError:
    print("No such nucleotide")


Enter letter: A
Adenine


#### Inspecting Exceptions with sys.exc_info()

In [1]:
# sys.exc_info() example
import sys

try:
    0 / 0
except:
    a, b, c = sys.exc_info()
    print(f"Error name: {a.__name__}")
    print(f"Message: {b}")
    print(f"Error in line: {c.tb_lineno}")


Error name: ZeroDivisionError
Message: division by zero
Error in line: 5


In [2]:
# File not found handling
import sys

try:
    x = open("random_filename")
except:
    a, b = sys.exc_info()[:2]
    print(f"Error name: {a.__name__}")
    print(f"Error code: {b.args[0]}")
    print(f"Error message: {b.args[1]}")


Error name: FileNotFoundError
Error code: 2
Error message: No such file or directory


#### Raising Exceptions
You can manually trigger exceptions using raise. This helps provide clearer error messages, especially during debugging.

- Bad example: 

In [ ]:
# Causes ZeroDivisionError
def avg(numbers):
    return sum(numbers) / len(numbers)

avg([])  # division by zero


In [ ]:
 avg([])

ZeroDivisionError: division by zero

- Improved version:

In [6]:
# Better error message using raise
def avg(numbers):
    if not numbers:
        raise ValueError("Please enter at least one element")
    return sum(numbers) / len(numbers)


In [7]:
avg([])
#ValueError: Please enter at least one element


ValueError: Please enter at least one element

### Creating Customized Exceptions 
One great feature of Python’s exception system is that you can define your own exceptions. read more about OOP in next chapter.

You can define a custom exception by subclassing the built-in Exception class.

- Example: Detecting invalid DNA sequences
Let’s define a new exception called NotDNAException. It should be raised when a DNA sequence contains a character other than 'a', 't', 'c', or 'g'.


In [8]:
# Custom exception class
#All Exceptions Derive from the Exception Class
class NotDNAException(Exception):
    """A user-defined exception"""
    def __init__(self, dna):
        self.dna = dna

    def __str__(self):
        # Return the first invalid character found
        for nt in self.dna:
            if nt not in 'atcg':
                return f"Invalid character: {nt}"
        return "Unknown DNA error"


In [9]:
# DNA sequence to check
dnaseq = 'agctwtacagt'

# Validate the sequence
if set(dnaseq) - set('atcg'):
    raise NotDNAException(dnaseq)
else:
    print('OK')


NotDNAException: Invalid character: w

### Fun & Practical Example
Suppose you're writing a program that asks users for their age. If someone types “twenty” instead of 20, your code will crash… unless you handle the error.

In [4]:
#funny
try:
    age = int(input("Enter your age: "))
    print(f"You are {age} years old.")
except ValueError:
    print("Oops! That doesn't look like a number.")


Enter your age:  twenty


Oops! That doesn't look like a number.


### Fun & Practical Example
Imagine a robot waiter asking how spicy you want your food. You answer “explosively spicy 🔥”. The robot wasn't ready for that.

In [5]:
def serve_food(spice_level):
    allowed = ['mild', 'medium', 'hot']
    if spice_level.lower() not in allowed:
        raise ValueError("This spice level is too dangerous for humans!")

try:
    user_input = input("Spice level (mild/medium/hot): ")
    serve_food(user_input)
    print("Serving your dish... 🌶️")
except ValueError as ve:
    print(f"Error: {ve}")


Spice level (mild/medium/hot):  atishi


Error: This spice level is too dangerous for humans!


### Theoretical Questions:

1. What is the meaning of LBYL and EAFP? Which one is used in Python?

2. What is an exception?

3. What is an “unhandled exception”?

4. When do you use finally and when do you use else?

5. Exceptions are often associated with file handling. Why?

6. How do you sort an error derived from a disk full condition from trying to write to a read-only file system?

7. Why is it not advisable to use except: to catch all kinds of exceptions, instead of using, for example, except IOError:?

8. Exceptions can be raised at will. Why would you do that?

9. What is the purpose of sys.exc_info()?

10. Explain the purpose of this function:

In [11]:
def formatExceptionInfo():
    """ Author: Arturo ’Buanzo’ Busleiman """
    cla, exc = sys.exc_info()[:2]
    excName = cla.__name__
    try:
        excArgs = exc.__dict__["args"]
    except KeyError:
        excArgs = str(exc)
    return (excName, excArgs)

### Code-Related Questions:

1. Write a function that asks the user to enter a number and handles the error if the input is not numeric.

2. Write a program that opens a file, reads its content, and gracefully handles the case where the file does not exist.

3. Write a custom exception NotDNAException that is raised when a string contains characters other than 'a', 't', 'c', 'g'.

4. Create a try...except...else...finally block and explain the flow of execution when there is an exception and when there isn’t.

5. Simulate a division by zero error and handle it using a proper exception block.

6. Write a function that takes a DNA sequence and translates it to RNA. Raise a custom exception if the input contains invalid characters.

7. Create a function that simulates reading from a biological data file. Handle FileNotFoundError and PermissionError appropriately.